### 1. Imports

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D

sns.set(style='white', context='notebook', palette='deep')

### 2. Data preparation

In [ ]:
# https://www.kaggle.com/competitions/digit-recognizer/data

# 2.1 Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X_train = train.drop(labels = ["label"], axis = 1)
Y_train = train["label"]

# 2.2 Check for null and missing values
X_train.isnull().any().describe()
test.isnull().any().describe()

# 2.3 Normalization
X_train = X_train / 255.0
test = test / 255.0

# 2.3 Reshape
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

# 2.4 Label encoding
Y_train = to_categorical(Y_train, num_classes = 10)

# 2.5 Split training and valdiation set
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=2)

### 3. CNN

In [ ]:
# 3.1 Define the model
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation = 'relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(Conv2D(filters=64, kernel_size = (3,3), padding = 'Same', activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

In [ ]:
# 3.2 Set the optimizer and annealer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer, loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# 3.3 Data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    feature_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening = False,
    rotation_range = 10,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = False,
    vertical_flip = False)

# Data augmentation Fit
datagen.fit(X_train)

In [ ]:
# Model Fit
epochs = 30
batch_size=86
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
                                            patience=3,
                                            verbose=1,
                                            factor=0.5,
                                            min_lr=0.00001)

history = model.fit_generator(
    datagen.flow(X_train, Y_train, batch_size=batch_size),
    epochs=epochs,
    validation_data=(X_val, Y_val),
    verbose = 2,
    steps_per_epoch=X_train.shape[0]//batch_size,
    callbacks=[learning_rate_reduction])

In [ ]:
# confusion matrix
Y_pred = model.predict(X_val)
Y_pred_classes = np.argmax(Y_pred,axis=1)
Y_true = np.argmax(Y_val, axis=1)
confusion_mtx = confusion_matrix(Y_true,Y_pred_classes)
f,ax = plt.subplots(figsize=(8,8))
sns.heatmap(confusion_mtx,annot=True, linewidths=0.01, cmap="Greens", linecolor="gray", fmt='.1f', ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
# Predict Final results
results = model.predict(test)